In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
pd.set_option('display.float_format', lambda x: '%.5f' % x)

### read files

#### campaign desc

In [2]:
camp_desc = pd.read_csv('campaign_desc.csv')

In [3]:
camp_desc.head()

,DESCRIPTION,CAMPAIGN,START_DAY,END_DAY
0,TypeB,24,659,719
1,TypeC,15,547,708
2,TypeB,25,659,691
3,TypeC,20,615,685
4,TypeB,23,646,684


In [4]:
len(camp_desc)

30

#### campaign table

In [5]:
camp_table = pd.read_csv('campaign_table.csv')

In [6]:
camp_table.head()

,DESCRIPTION,household_key,CAMPAIGN
0,TypeA,17,26
1,TypeA,27,26
2,TypeA,212,26
3,TypeA,208,26
4,TypeA,192,26


In [7]:
len(camp_table)

7208

#### causal data

In [8]:
causal = pd.read_csv('causal_data.csv')

In [9]:
causal.head()

,PRODUCT_ID,STORE_ID,WEEK_NO,display,mailer
0,26190,286,70,0,A
1,26190,288,70,0,A
2,26190,289,70,0,A
3,26190,292,70,0,A
4,26190,293,70,0,A


In [10]:
len(causal)

36786524

#### coupon

In [11]:
coupon = pd.read_csv('coupon.csv')

In [12]:
coupon.head()

,COUPON_UPC,PRODUCT_ID,CAMPAIGN
0,10000089061,27160,4
1,10000089064,27754,9
2,10000089073,28897,12
3,51800009050,28919,28
4,52100000076,28929,25


In [13]:
len(coupon)

124548

#### coupon redempt

In [14]:
coupon_redmpt = pd.read_csv('coupon_redempt.csv')

In [15]:
coupon_redmpt.head()

,household_key,DAY,COUPON_UPC,CAMPAIGN
0,1,421,10000085364,8
1,1,421,51700010076,8
2,1,427,54200000033,8
3,1,597,10000085476,18
4,1,597,54200029176,18


#### hh_demographic

In [16]:
hh_demog = pd.read_csv('hh_demographic.csv')

In [17]:
hh_demog.head()

,AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,HOMEOWNER_DESC,HH_COMP_DESC,HOUSEHOLD_SIZE_DESC,KID_CATEGORY_DESC,household_key
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13
4,45-54,B,50-74K,Homeowner,Single Female,1,None/Unknown,16


In [18]:
len(hh_demog)

801

#### product

In [19]:
product = pd.read_csv('product.csv')

In [20]:
product.head()

,PRODUCT_ID,MANUFACTURER,DEPARTMENT,BRAND,COMMODITY_DESC,SUB_COMMODITY_DESC,CURR_SIZE_OF_PRODUCT
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ


In [21]:
len(product)

92353

#### transaction_data

In [22]:
transaction_data = pd.read_csv('transaction_data.csv')

In [23]:
transaction_data.head()

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC
0,2375,26984851472,1,1004906,1,1.39000,364,-0.60000,1631,1,0.00000,0.00000
1,2375,26984851472,1,1033142,1,0.82000,364,0.00000,1631,1,0.00000,0.00000
2,2375,26984851472,1,1036325,1,0.99000,364,-0.30000,1631,1,0.00000,0.00000
3,2375,26984851472,1,1082185,1,1.21000,364,0.00000,1631,1,0.00000,0.00000
4,2375,26984851472,1,8160430,1,1.50000,364,-0.39000,1631,1,0.00000,0.00000


In [24]:
len(transaction_data)

2595732

#### comment : reading files finished 

### ***

## I will start from transaction data

### lets find unique households in transaction data

In [25]:
transaction_data['household_key'].nunique()

2500

### now, lets find if there is a zero quantity rows in our data, if there is , I will discard them

In [26]:
len(transaction_data[transaction_data['QUANTITY'] == 0])

14466

In [27]:
transaction_data2 = transaction_data[transaction_data['QUANTITY']!=0]

### calculate actual prices

In [28]:
transaction_data2['actual_product_price'] = np.where(transaction_data2['RETAIL_DISC']>0,
                                            (transaction_data2['SALES_VALUE'] - (transaction_data2['RETAIL_DISC'] - transaction_data2['COUPON_MATCH_DISC'])) / transaction_data2['QUANTITY'],
                                           np.where(transaction_data2['RETAIL_DISC']==0,
                                            (transaction_data2['SALES_VALUE'] - transaction_data2['COUPON_MATCH_DISC']) / transaction_data2['QUANTITY'],transaction_data2['SALES_VALUE']))

C:\Users\Hincal\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [29]:
len(transaction_data2)

2581266

### find how many distinct product we have in our transaction data

In [30]:
transaction_data['PRODUCT_ID'].nunique()

92339

In [31]:
product['PRODUCT_ID'].nunique()

92353

In [32]:
len(transaction_data2[transaction_data2['PRODUCT_ID'].isin (product['PRODUCT_ID'])==True])

2581266

#### comment : it is good, we can merge transaction data with product data

### So lets merge !

In [33]:
product.reset_index(inplace=True)

In [34]:
transaction_data3 = pd.merge(transaction_data2,product,on='PRODUCT_ID',how='left')
transaction_data3.reset_index(inplace=True)

In [35]:
len(transaction_data3)

2581266

In [36]:
transaction_data3.isnull().sum()

level_0                 0
household_key           0
BASKET_ID               0
DAY                     0
PRODUCT_ID              0
QUANTITY                0
SALES_VALUE             0
STORE_ID                0
RETAIL_DISC             0
TRANS_TIME              0
WEEK_NO                 0
COUPON_DISC             0
COUPON_MATCH_DISC       0
actual_product_price    0
index                   0
MANUFACTURER            0
DEPARTMENT              0
BRAND                   0
COMMODITY_DESC          0
SUB_COMMODITY_DESC      0
CURR_SIZE_OF_PRODUCT    0
dtype: int64

In [37]:
transaction_data3.head()

,level_0,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,...,COUPON_DISC,COUPON_MATCH_DISC,actual_product_price,index,MANUFACTURER,DEPARTMENT,BRAND,COMMODITY_DESC,SUB_COMMODITY_DESC,CURR_SIZE_OF_PRODUCT
0,0,2375,26984851472,1,1004906,1,1.39000,364,-0.60000,1631,...,0.00000,0.00000,1.39000,26869,69,PRODUCE,Private,POTATOES,POTATOES RUSSET (BULK&BAG),5 LB
1,1,2375,26984851472,1,1033142,1,0.82000,364,0.00000,1631,...,0.00000,0.00000,0.82000,30031,2,PRODUCE,National,ONIONS,ONIONS SWEET (BULK&BAG),40 LB
2,2,2375,26984851472,1,1036325,1,0.99000,364,-0.30000,1631,...,0.00000,0.00000,0.99000,30387,69,PRODUCE,Private,VEGETABLES - ALL OTHERS,CELERY,
3,3,2375,26984851472,1,1082185,1,1.21000,364,0.00000,1631,...,0.00000,0.00000,1.21000,35576,2,PRODUCE,National,TROPICAL FRUIT,BANANAS,40 LB
4,4,2375,26984851472,1,8160430,1,1.50000,364,-0.39000,1631,...,0.00000,0.00000,1.50000,63317,69,PRODUCE,Private,ORGANICS FRUIT & VEGETABLES,ORGANIC CARROTS,1 LB


#### I will merge camp and camp desc tables now

In [41]:
camp_merged = pd.merge(camp_table,camp_desc,how='left',on=['CAMPAIGN','DESCRIPTION'])

In [39]:
len(camp_merged)

7208

In [42]:
camp_merged.head()

,DESCRIPTION,household_key,CAMPAIGN,START_DAY,END_DAY
0,TypeA,17,26,224,264
1,TypeA,27,26,224,264
2,TypeA,212,26,224,264
3,TypeA,208,26,224,264
4,TypeA,192,26,224,264


In [43]:
camp_merged.isnull().sum()

DESCRIPTION      0
household_key    0
CAMPAIGN         0
START_DAY        0
END_DAY          0
dtype: int64

### Now I will join transaction data with camp data

In [44]:
transaction_data4 = pd.merge(transaction_data3,camp_merged,how='left',on='household_key')

In [45]:
len(transaction_data3)

2581266

In [46]:
transaction_data4.head()

,level_0,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,...,MANUFACTURER,DEPARTMENT,BRAND,COMMODITY_DESC,SUB_COMMODITY_DESC,CURR_SIZE_OF_PRODUCT,DESCRIPTION,CAMPAIGN,START_DAY,END_DAY
0,0,2375,26984851472,1,1004906,1,1.39000,364,-0.60000,1631,...,69,PRODUCE,Private,POTATOES,POTATOES RUSSET (BULK&BAG),5 LB,TypeA,13.00000,504.00000,551.00000
1,0,2375,26984851472,1,1004906,1,1.39000,364,-0.60000,1631,...,69,PRODUCE,Private,POTATOES,POTATOES RUSSET (BULK&BAG),5 LB,TypeA,18.00000,587.00000,642.00000
2,1,2375,26984851472,1,1033142,1,0.82000,364,0.00000,1631,...,2,PRODUCE,National,ONIONS,ONIONS SWEET (BULK&BAG),40 LB,TypeA,13.00000,504.00000,551.00000
3,1,2375,26984851472,1,1033142,1,0.82000,364,0.00000,1631,...,2,PRODUCE,National,ONIONS,ONIONS SWEET (BULK&BAG),40 LB,TypeA,18.00000,587.00000,642.00000
4,2,2375,26984851472,1,1036325,1,0.99000,364,-0.30000,1631,...,69,PRODUCE,Private,VEGETABLES - ALL OTHERS,CELERY,,TypeA,13.00000,504.00000,551.00000


In [47]:
transaction_data4.isnull().sum()

level_0                      0
household_key                0
BASKET_ID                    0
DAY                          0
PRODUCT_ID                   0
QUANTITY                     0
SALES_VALUE                  0
STORE_ID                     0
RETAIL_DISC                  0
TRANS_TIME                   0
WEEK_NO                      0
COUPON_DISC                  0
COUPON_MATCH_DISC            0
actual_product_price         0
index                        0
MANUFACTURER                 0
DEPARTMENT                   0
BRAND                        0
COMMODITY_DESC               0
SUB_COMMODITY_DESC           0
CURR_SIZE_OF_PRODUCT         0
DESCRIPTION             302298
CAMPAIGN                302298
START_DAY               302298
END_DAY                 302298
dtype: int64

### mark if transaction day is between campaign dates

In [48]:
transaction_data4['is_between_camp_dates'] = np.where((transaction_data4['DAY']>=transaction_data4['START_DAY']) & (transaction_data4['DAY']<=transaction_data4['END_DAY']),1,0)

In [49]:
transaction_data4.head(21)

,level_0,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,...,DEPARTMENT,BRAND,COMMODITY_DESC,SUB_COMMODITY_DESC,CURR_SIZE_OF_PRODUCT,DESCRIPTION,CAMPAIGN,START_DAY,END_DAY,is_between_camp_dates
0,0,2375,26984851472,1,1004906,1,1.39000,364,-0.60000,1631,...,PRODUCE,Private,POTATOES,POTATOES RUSSET (BULK&BAG),5 LB,TypeA,13.00000,504.00000,551.00000,0
1,0,2375,26984851472,1,1004906,1,1.39000,364,-0.60000,1631,...,PRODUCE,Private,POTATOES,POTATOES RUSSET (BULK&BAG),5 LB,TypeA,18.00000,587.00000,642.00000,0
2,1,2375,26984851472,1,1033142,1,0.82000,364,0.00000,1631,...,PRODUCE,National,ONIONS,ONIONS SWEET (BULK&BAG),40 LB,TypeA,13.00000,504.00000,551.00000,0
3,1,2375,26984851472,1,1033142,1,0.82000,364,0.00000,1631,...,PRODUCE,National,ONIONS,ONIONS SWEET (BULK&BAG),40 LB,TypeA,18.00000,587.00000,642.00000,0
4,2,2375,26984851472,1,1036325,1,0.99000,364,-0.30000,1631,...,PRODUCE,Private,VEGETABLES - ALL OTHERS,CELERY,,TypeA,13.00000,504.00000,551.00000,0
5,2,2375,26984851472,1,1036325,1,0.99000,364,-0.30000,1631,...,PRODUCE,Private,VEGETABLES - ALL OTHERS,CELERY,,TypeA,18.00000,587.00000,642.00000,0
6,3,2375,26984851472,1,1082185,1,1.21000,364,0.00000,1631,...,PRODUCE,National,TROPICAL FRUIT,BANANAS,40 LB,TypeA,13.00000,504.00000,551.00000,0
7,3,2375,26984851472,1,1082185,1,1.21000,364,0.00000,1631,...,PRODUCE,National,TROPICAL FRUIT,BANANAS,40 LB,TypeA,18.00000,587.00000,642.00000,0
8,4,2375,26984851472,1,8160430,1,1.50000,364,-0.39000,1631,...,PRODUCE,Private,ORGANICS FRUIT & VEGETABLES,ORGANIC CARROTS,1 LB,TypeA,13.00000,504.00000,551.00000,0
9,4,2375,26984851472,1,8160430,1,1.50000,364,-0.39000,1631,...,PRODUCE,Private,ORGANICS FRUIT & VEGETABLES,ORGANIC CARROTS,1 LB,TypeA,18.00000,587.00000,642.00000,0


In [50]:
transaction_data4.isnull().sum()

level_0                       0
household_key                 0
BASKET_ID                     0
DAY                           0
PRODUCT_ID                    0
QUANTITY                      0
SALES_VALUE                   0
STORE_ID                      0
RETAIL_DISC                   0
TRANS_TIME                    0
WEEK_NO                       0
COUPON_DISC                   0
COUPON_MATCH_DISC             0
actual_product_price          0
index                         0
MANUFACTURER                  0
DEPARTMENT                    0
BRAND                         0
COMMODITY_DESC                0
SUB_COMMODITY_DESC            0
CURR_SIZE_OF_PRODUCT          0
DESCRIPTION              302298
CAMPAIGN                 302298
START_DAY                302298
END_DAY                  302298
is_between_camp_dates         0
dtype: int64

#### to here it is ok *

### join coupon tables

In [51]:
coupon_redmpt.head()

,household_key,DAY,COUPON_UPC,CAMPAIGN
0,1,421,10000085364,8
1,1,421,51700010076,8
2,1,427,54200000033,8
3,1,597,10000085476,18
4,1,597,54200029176,18


In [53]:
len(coupon_redmpt)

2318

In [52]:
coupon.head()

,COUPON_UPC,PRODUCT_ID,CAMPAIGN
0,10000089061,27160,4
1,10000089064,27754,9
2,10000089073,28897,12
3,51800009050,28919,28
4,52100000076,28929,25


In [54]:
len(coupon)

124548

In [55]:
coupon_merged =  pd.merge(coupon,coupon_redmpt,how='left',on=['CAMPAIGN','COUPON_UPC'])

In [57]:
coupon_merged2 = coupon_merged[coupon_merged['household_key'].isnull()==False]

In [58]:
coupon_merged2.head()

,COUPON_UPC,PRODUCT_ID,CAMPAIGN,household_key,DAY
1,10000089064,27754,9,321.00000,446.00000
2,10000089064,27754,9,1773.00000,439.00000
3,10000089073,28897,12,1229.00000,491.00000
4,10000089073,28897,12,1633.00000,497.00000
5,10000089073,28897,12,1856.00000,492.00000


### **

### Now I will find each household's daily-weekly aggregated buying information

In [ ]:
agg1 = transaction_data2.groupby(['household_key','DAY','WEEK_NO']).agg({'BASKET_ID' : 'count',
                                                       'PRODUCT_ID': 'count',
                                                       'QUANTITY': 'sum',
                                                       'SALES_VALUE':['sum','mean'],
                                                       'STORE_ID' : 'count',
                                                       'RETAIL_DISC' : ['sum','mean'],
                                                       'TRANS_TIME': 'mean',
                                                       'COUPON_DISC': ['min','max','mean'],
                                                       'COUPON_MATCH_DISC': ['min','max','mean'],
                                                       'actual_product_price' : 'mean'
    
})

agg1.columns = agg1.columns.droplevel(0)

agg1.columns = ['basket_cnt','product_cnt','sum_quantity','sum_sales','mean_sales','store_cnt','retail_disc_sum',
                'retail_disc_mean','avg_trans_time','min_coupon_disc','max_coupon_disc','avg_coupon_disc',
                'min_coup_mdisc','max_coup_mdisc','avg_coup_mdisc','avg_actual_prod_price']

agg1.reset_index(inplace=True)

agg1.head()

In [ ]:
agg1[agg1['household_key']==1]